In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os
import json

In [2]:
user_agent = {'User-agent': 'Mozilla/5.0'}

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [3]:
def get_film_data(film_id):
    
    delimiter = '+_+'
    
    film_url = "http://www.imdb.com/title/" + film_id
    driver.get(film_url)
    
    response = requests.get(film_url, headers = user_agent)
    html = response.text
    soup = bs(html, 'lxml')
    
    film_dict = {}
    
    try:
        genre_selector = "//*[@id='titleStoryLine']//a[contains(@href, '/genre/')]"
        film_dict['genres'] = ''
        for genre in driver.find_elements_by_xpath(genre_selector):
            film_dict['genres'] = film_dict['genres'] + delimiter + genre.text
    except NoSuchElementException:
        film_dict['genres'] = None
        
    try:
        title_selector = "//*[@id='title-overview-widget']//*[@itemprop='name']"
        film_dict['title'] = driver.find_element_by_xpath(title_selector).text
    except NoSuchElementException:
        film_dict['title'] = None
    
    try:
        year_selector = '//*[@id="titleYear"]'
        film_dict['year'] = driver.find_element_by_xpath(year_selector).text.split('(')[1].split(')')[0]
    except NoSuchElementException:
        film_dict['year'] = None  
    
    try:
        rating_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingValue']"
        film_dict['rating'] = driver.find_element_by_xpath(rating_selector).text
    except NoSuchElementException:
        film_dict['rating'] = None  
   
    try:
        votes_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingCount']"
        film_dict['votes'] = driver.find_element_by_xpath(votes_selector).text
    except NoSuchElementException:
        film_dict['votes'] = None     
    
    try:
        directors_selector = "//*[@id='title-overview-widget']//*[@itemprop='director']"
        film_dict['directors'] = ''
        for director in driver.find_elements_by_xpath(directors_selector):
            film_dict['directors'] = film_dict['directors'] + delimiter + director.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['directors'] = None    
  
    try:
        writers_selector = "//*[@id='title-overview-widget']//*[@itemprop='creator']"
        film_dict['writers'] = ''
        for writer in driver.find_elements_by_xpath(writers_selector):
            film_dict['writers'] = film_dict['writers'] + delimiter + writer.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['writers'] = None     

    try:
        actors_selector = "//*[@id='titleCast']//*[@itemprop='name']"
        film_dict['actors'] = ''
        for actor in driver.find_elements_by_xpath(actors_selector):
            film_dict['actors'] = film_dict['actors'] + delimiter + actor.text
    except NoSuchElementException:
        film_dict['actors'] = None          

    try:
        keywords_selector = "//*[@class='see-more inline canwrap']//*[@itemprop='keywords']"
        film_dict['keywords'] = ''
        for keyword in driver.find_elements_by_xpath(keywords_selector):
            film_dict['keywords'] = film_dict['keywords'] + delimiter + keyword.text
    except NoSuchElementException:
        film_dict['keywords'] = None       

    try:
        mpaa_selector = "//meta[@itemprop='contentRating']"
        film_dict['mpaa'] = driver.find_element_by_xpath(mpaa_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['mpaa'] = None      

    try:
        release_date_selector = "//meta[@itemprop='datePublished']"
        film_dict['release_date'] = driver.find_element_by_xpath(release_date_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['release_date'] = None     

    try:
        languages_selector = "//*[@id='titleDetails']//a[contains(@href, 'language')]"
        film_dict['languages'] = ''
        for language in driver.find_elements_by_xpath(languages_selector):
            film_dict['languages'] = film_dict['languages'] + delimiter + language.text
    except NoSuchElementException:
        film_dict['languages'] = None   
    
    try:
        countries_selector = "//*[@id='titleDetails']//a[contains(@href, 'country_of_origin')]"
        film_dict['countries'] = ''
        for country in driver.find_elements_by_xpath(countries_selector):
            film_dict['countries'] = film_dict['countries'] + delimiter + country.text
    except NoSuchElementException:
        film_dict['countries'] = None 
    
    try:
        runtime_selector = "//*[@id='titleDetails']//*[@itemprop='duration']"
        film_dict['runtime'] = int(driver.find_element_by_xpath(runtime_selector).text.split(' ')[0])
    except NoSuchElementException:
        film_dict['runtime'] = None 
     
    film_dict['budget'] = None
    film_dict['gross_usa'] = None
    for h4 in soup.find_all('h4'):
        if "Budget:" in h4:
            film_dict['budget'] = h4.next_sibling        
        if "Gross USA:" in h4:
            film_dict['gross_usa'] = h4.next_sibling
            
    return film_dict

In [4]:
with open("film_ids.json","r") as f:
      data = f.read()
        
film_ids = json.loads(data)

In [5]:
dict_list = []
dummy = 0

for film_id in film_ids[8000:9000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
import pandas as pd

data = json.dumps(dict_list)

with open("film_data_9.json","w") as f:
      f.write(data)

with open("film_data_9.json","r") as f:
      data = f.read()
        
d = json.loads(data)

df = pd.DataFrame(d)

df

,actors,budget,countries,directors,genres,gross_usa,keywords,languages,mpaa,rating,release_date,runtime,title,votes,writers,year
0,+_+Ellen Barkin+_+Wendy Crewson+_+Peta Wilson+...,None,+_+USA,+_+Damian Harris,+_+Mystery+_+Thriller,None,+_+lesbianism+_+bondage+_+labia+_+nude in a wi...,+_+English,R,5.4,2000-02-11,117.0,Mercy (2000),"1,967",+_+David L. Lindsey+_+Damian Harris,2000
1,+_+Callie Thorne+_+Bill Sage+_+Noah Fleiss+_+R...,None,+_+USA,+_+Stephen Kinsella,+_+Comedy+_+Drama,None,+_+teenager+_+teenage boy+_+parking meter+_+mo...,+_+English,NOT RATED,6.5,2000-01-26,98.0,Double Parked (2000),135,"+_+Stephen Kinsella,+_+Paul Solberg",2000
2,+_+Bob Jay Mills+_+Al Schuermann+_+Gregorio+_+...,None,+_+USA,+_+Frank Novak,+_+Comedy,None,+_+feature film directorial debut+_+directoria...,+_+English,R,6.9,2000-11-11,NaN,Better Housekeeping (2000),74,+_+Frank Novak,2000
3,+_+Tommy Tiernan+_+Kate Hudson+_+Frances O'Con...,None,+_+Ireland+_+UK+_+USA,+_+Gerard Stembridge,+_+Comedy+_+Romance,"$151,559,",+_+alliteration in title+_+name in title+_+mal...,+_+English+_+French,R,5.9,2001-01-19,97.0,About Adam (2000),"5,969","+_+Gerard Stembridge,+_+Tommy Tiernan",2000
4,+_+Dwight Yoakam+_+Vince Vaughn+_+Billy Bob Th...,"$4,000,000\n",+_+USA,+_+Dwight Yoakam,+_+Drama+_+Western,"$16,609,",+_+revisionist western+_+desert+_+christmas ev...,+_+English,R,4.2,2000-01-28,104.0,"South of Heaven, West of Hell (2000)","1,264","+_+Stan Bertheaud,+_+Otto Felix",2000
5,+_+Bette Midler+_+Nathan Lane+_+Stockard Chann...,"$44,000,000\n",+_+USA+_+UK+_+Germany+_+Japan+_+Canada,+_+Andrew Bergman,+_+Biography+_+Comedy+_+Romance,"$2,962,465\n",+_+radio show+_+singer+_+blonde+_+friend+_+diner,+_+English,R,5.4,2000-01-28,95.0,Isn't She Great (2000),"2,082",+_+Michael Korda+_+Paul Rudnick,2000
6,+_+David F. Camoin+_+Marie Watkins Crocker+_+M...,"$7,000\n",+_+USA,+_+Thomas Camoin,+_+Comedy+_+Drama,None,+_+writing+_+drugs+_+cannabis+_+independent film,+_+English,None,4.9,2000-01-28,NaN,How to Smoke Pot (2000),10,+_+Thomas Camoin,2000
7,+_+Chris Elliott+_+Mark Webber+_+Jean Smart+_+...,"$13,000,000\n",+_+USA,+_+Chris Koch,+_+Adventure+_+Comedy+_+Family,"$60,020,107\n",+_+meteorologist+_+snowed in+_+snowplow+_+weat...,+_+English,PG,5.0,2000-02-11,89.0,Snow Day (2000),"10,129","+_+Will McRobb,+_+Chris Viscardi",2000
8,+_+Ben Weber+_+Timothy Olyphant+_+Matt McGrath...,"$1,000,000\n",+_+USA,+_+Greg Berlanti,+_+Comedy+_+Drama+_+Romance+_+Sport,"$1,744,858,",+_+male objectification+_+bare chested male+_+...,+_+English,R,7.0,2000-10-20,94.0,The Broken Hearts Club: A Romantic Comedy (2000),"6,404",+_+Greg Berlanti,2000
9,+_+Courtney Love+_+Kiefer Sutherland+_+Lisa Sh...,None,+_+USA,+_+Gary Walkow,+_+Drama,None,+_+writer+_+hairy chest+_+bare chested male+_+...,+_+English,R,5.6,2000-01-29,93.0,Beat (2000),"1,353",+_+Gary Walkow,2000
